In [1]:
import copy

'''
SIMULATE_POPULATION.PY: Simulate a population of cells with the Punisher and a burdensome gene
'''
# By Kirill Sechkar

# PACKAGE IMPORTS ------------------------------------------------------------------------------------------------------
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB

# set up bokeh output to notebook
bkio.reset_output()
bkio.output_notebook() 

import time

# import os
# import multiprocessing
# os.environ["XLA_FLAGS"] = "-- python==3.9xla_force_host_platform_device_count={}".format(multiprocessing.cpu_count())

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# CIRCUIT AND EXTERNAL INPUT IMPORTS -----------------------------------------------------------------------------------
from cell_model import *
import synthetic_circuits as circuits

# population simulator
from pop_simulator import *

Loading BokehJS ...

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


In [2]:
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.punisher_b_initialise,
    circuits.punisher_b_ode,
    circuits.punisher_b_F_calc,
    par, init_conds)  # load the circuit

In [3]:
# PARAMETERISE THE CIRCUIT

# BURDENSOME SYNTHETIC GENE
par['c_b'] = 1
par['a_b'] = 1e5

# PUNISHER
# switch gene conc
par['c_switch'] = 1.0  # gene concentration (nM)
par['a_switch'] = 4000.0  # promoter strength (unitless)
par['d_switch']=0.01836
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch']/80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0#0.01836 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
init_conds['cat_pb'] = 1.0  # gene concentration (nM) - INITIAL CONDITION< NOT PARAMETER as it can be cut out by the integrase
par['a_cat'] = 5000.0  # promoter strength (unitless)
# synthetic protease gene
par['c_prot'] = 1.0  # gene concentration (nM)
par['a_prot'] = 250.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0 # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['baseline_switch_alt'] = 0
par['p_switch_ac_frac'] = 0.85  # active fraction of protein (i.e. share of molecules bound by the inducer)

# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 10.5 * (10.0 ** 3)

In [4]:
# INITIALISE DICTIONARIES OF CELLULAR VARIABLE VALUES IN DIFFERENT STATES

# cell growth rates
growth_rates={
    'BSPC': {'H': 0, 'L': 0, '0':0},
    'BOPC': {'H': 0, 'L': 0, '0':0},
    'BSOC': {'H': 0, 'L': 0, '0':0},
    'BOOC': {'H': 0, 'L': 0, '0':0},
    'OSPC': {'H': 0, 'L': 0, '0':0},
    'OOPC': {'H': 0, 'L': 0, '0':0},
    'OSOC': {'H': 0, 'L': 0, '0':0},
    'OOOC': {'H': 0, 'L': 0, '0':0},
    'BSPO': {'H': 0, 'L': 0, '0':0},
    'BOPO': {'H': 0, 'L': 0, '0':0},
    'BSOO': {'H': 0, 'L': 0, '0':0},
    'BOOO': {'H': 0, 'L': 0, '0':0},
    'OSPO': {'H': 0, 'L': 0, '0':0},
    'OOPO': {'H': 0, 'L': 0, '0':0},
    'OSOO': {'H': 0, 'L': 0, '0':0},
    'OOOO': {'H': 0, 'L': 0, '0':0}
}
# record integrase activity rates
integrase_activities=copy.deepcopy(growth_rates)

# burdensome protein levels
p_bs=copy.deepcopy(growth_rates)

In [5]:
# DETERMINISTIC SIMULATION - WITH CAT PRESENT, FIND ALL TRUE SWITCH EQUILIBRIA FOR ALL GENETIC STATES
# SET SIMULATION PARAMETERS
# diffrax simulator
savetimestep = 1  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver
# simulation time frames
tf = (0,50) # time frame for simulation before burdensome gene loss

# BSPC: H
#simulate
par_bspc=par.copy()
par_bspc['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
init_conds['m_switch']=3000
sol = ode_sim(par_bspc,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_bspc, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_bspc, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_bspc_h = l  # growth rate
p_b_bspc_h =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
intact_bspc_h = intact_calc(xs[-1,circuit_name2pos['p_int']],l,par,init_conds['cat_pb'])
intconc_bspc_h = xs[-1,circuit_name2pos['p_int']]

# BSPC: L
#simulate
init_conds['m_switch']=0
sol = ode_sim(par_bspc,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_bspc, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_bspc, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_bspc_l = l  # growth rate
p_b_bspc_l =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
intact_bspc_l = intact_calc(xs[-1,circuit_name2pos['p_int']],l,par,init_conds['cat_pb'])
intconc_bspc_l = xs[-1,circuit_name2pos['p_int']]

# OSPC: H
#simulate
par_ospc=par.copy()
par_ospc['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_ospc['func_b'] = 0
init_conds['m_switch']=3000
sol = ode_sim(par_ospc,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_ospc, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_bspc, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_ospc_h = l  # growth rate
p_b_ospc_h =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
intact_ospc_h = intact_calc(xs[-1,circuit_name2pos['p_int']],l,par,init_conds['cat_pb'])
intconc_ospc_h = xs[-1,circuit_name2pos['p_int']]

# BOPC: 0
#simulate
par_bopc=par.copy()
par_bopc['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_bopc['func_switch'] = 0
init_conds['m_switch']=0
sol = ode_sim(par_bopc,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_bopc, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_bopc, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_bopc_0 = l  # growth rate
p_b_bopc_0 =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
intact_bopc_0 = intact_calc(xs[-1,circuit_name2pos['p_int']],l,par,init_conds['cat_pb'])
intconc_bopc_0 = xs[-1,circuit_name2pos['p_int']]

# OOPC: 0
#simulate
par_oopc=par.copy()
par_oopc['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_oopc['func_b'] = 0
par_oopc['func_switch'] = 0
init_conds['m_switch']=0
sol = ode_sim(par_oopc,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_oopc, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_oopc, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_oopc_0 = l  # growth rate
p_b_oopc_0 =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
intact_oopc_0 = intact_calc(xs[-1,circuit_name2pos['p_int']],l,par,init_conds['cat_pb'])
intconc_oopc_0 = xs[-1,circuit_name2pos['p_int']]

# BSOC: H
#simulate
par_bsoc=par.copy()
par_bsoc['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_bsoc['func_prot'] = 0
init_conds['m_switch']=3000.0
sol = ode_sim(par_bsoc,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_bsoc, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_bsoc, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_bsoc_h = l  # growth rate
p_b_bsoc_h =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
intact_bsoc_h = intact_calc(xs[-1,circuit_name2pos['p_int']],l,par,init_conds['cat_pb'])
intconc_bsoc_h = xs[-1,circuit_name2pos['p_int']]

# OSOC:H
#simulate
par_osoc=par.copy()
par_osoc['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_osoc['func_b'] = 0
par_osoc['func_prot'] = 0
init_conds['m_switch']=3000
sol = ode_sim(par_osoc,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_osoc, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_osoc, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_osoc_h = l  # growth rate
p_b_osoc_h =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
intact_osoc_h = intact_calc(xs[-1,circuit_name2pos['p_int']],l,par,init_conds['cat_pb'])
intconc_osoc_h = xs[-1,circuit_name2pos['p_int']]

# BOOC: 0
#simulate
par_booc=par.copy()
par_booc['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_booc['func_switch'] = 0
par_booc['func_prot'] = 0
init_conds['m_switch']=0
sol = ode_sim(par_booc,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_booc, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_booc, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_booc_0 = l  # growth rate
p_b_booc_0 =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
intact_booc_0 = intact_calc(xs[-1,circuit_name2pos['p_int']],l,par,init_conds['cat_pb'])
intconc_booc_0 = xs[-1,circuit_name2pos['p_int']]

# OOOC: 0
#simulate
par_oooc=par.copy()
par_oooc['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_oooc['func_b'] = 0
par_oooc['func_switch'] = 0
par_oooc['func_prot'] = 0
init_conds['m_switch']=0
sol = ode_sim(par_oooc,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_oooc, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_oooc, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_oooc_0 = l  # growth rate
p_b_oooc_0 =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
intact_oooc_0 = intact_calc(xs[-1,circuit_name2pos['p_int']],l,par,init_conds['cat_pb'])
intconc_oooc_0 = xs[-1,circuit_name2pos['p_int']]

/mnt/c/Users/ersat/CODE/punisher/Fig5/pop_simulator.py:674: RuntimeWarning: divide by zero encountered in scalar divide
  exp_time_to_all_cutouts=(1/(intact_rate)) * np.sum(1/np.arange(1,c_cat_pb_int+0.1,1)) # expected time to all cutouts - i.e. expected maximimum of c_cat exp. dist. samples


In [6]:
print(l_bspc_h, p_b_bspc_h, intconc_bspc_h)
print(l_bspc_l, p_b_bspc_l, intconc_bspc_l)
print(l_ospc_h, p_b_ospc_h, intconc_ospc_h)
print(l_bopc_0, p_b_bopc_0, intconc_bopc_0)
print(l_oopc_0, p_b_oopc_0, intconc_oopc_0)
print(l_bsoc_h, p_b_bsoc_h, intconc_bsoc_h)
print(l_osoc_h, p_b_osoc_h, intconc_osoc_h)
print(l_booc_0, p_b_booc_0, intconc_booc_0)
print(l_oooc_0, p_b_oooc_0, intconc_oooc_0)

1.3337455813077181 203005.79363521142 87.76746499884888
1.3364265287204893 203112.00510247593 3.519842478581789
1.52511943387877 0.0 104.55510433630727
1.3365387347959332 203116.44864442173 0.0
1.5289245468430066 0.0 0.0
1.3302743206926888 202164.1733972055 100.89400645804247
1.5207280246334085 0.0 115.39340961218083
1.3369592964431116 203180.3796178491 0.0
1.5294749609531855 0.0 0.0


In [7]:
# FOR CAT GENE PRESENT, RECORD GROWTH RATES, BURDENSOME PROTEIN LEVELS AND INTEGRASE ACTIVITIES FOR ALL STATES

# BSPC: H
growth_rates['BSPC']['H'] = l_bspc_h
p_bs['BSPC']['H'] = p_b_bspc_h
integrase_activities['BSPC']['H'] = intact_bspc_h

# BSPC: L
growth_rates['BSPC']['L'] = l_bspc_l
p_bs['BSPC']['L'] = p_b_bspc_l
integrase_activities['BSPC']['L'] = intact_bspc_l

# BSPC:0
# not considered

# OSPC: H
growth_rates['OSPC']['H'] = l_ospc_h
p_bs['OSPC']['H'] = 0
integrase_activities['OSPC']['H'] = intact_ospc_h

# OSPC:L
growth_rates['OSPC']['L'] = l_ospc_h # same as OSPC:H
p_bs['OSPC']['L'] = 0
integrase_activities['OSPC']['L'] = intact_bspc_l

# OSPC:0
# not considered

# BOPC: H
growth_rates['BOPC']['H'] = l_bopc_0
p_bs['BOPC']['H'] = p_b_bopc_0
integrase_activities['BOPC']['H'] = intact_bspc_h

# BOPC: L
growth_rates['BOPC']['L'] = l_bopc_0
p_bs['BOPC']['L'] = p_b_bopc_0
integrase_activities['BOPC']['L'] = intact_bspc_l

# BOPC: 0
growth_rates['BOPC']['0'] = l_bopc_0
p_bs['BOPC']['0'] = p_b_bopc_0
integrase_activities['BOPC']['0'] = 0

# OOPC: H
growth_rates['OOPC']['H'] = l_oopc_0
p_bs['OOPC']['H'] = 0
integrase_activities['OOPC']['H'] = intact_ospc_h

# OOPC: L
growth_rates['OOPC']['L'] = l_oopc_0
p_bs['OOPC']['L'] = 0
integrase_activities['OOPC']['L'] = intact_bspc_l

# OOPC: 0
growth_rates['OOPC']['0'] = l_oopc_0
p_bs['OOPC']['0'] = 0
integrase_activities['OOPC']['0'] = 0

# BSOC: H
growth_rates['BSOC']['H'] = l_bsoc_h
p_bs['BSOC']['H'] = p_b_bsoc_h
integrase_activities['BSOC']['H'] = intact_bsoc_h

# BSOC: L
growth_rates['BSOC']['L'] = l_bsoc_h # AMENDED
p_bs['BSOC']['L'] = p_b_bsoc_h  # AMENDED
integrase_activities['BSOC']['L'] = intact_bspc_l

# BSOC: 0
# not considered

# OSOC: H
growth_rates['OSOC']['H'] = l_osoc_h
p_bs['OSOC']['H'] = 0
integrase_activities['OSOC']['H'] = intact_osoc_h

# OSOC: L
growth_rates['OSOC']['L'] = l_osoc_h # AMENDED
p_bs['OSOC']['L'] = 0
integrase_activities['OSOC']['L'] = intact_bspc_l

# OSOC: 0
# not considered

# BOOC: H
growth_rates['BOOC']['H'] = l_booc_0
p_bs['BOOC']['H'] = p_b_booc_0
integrase_activities['BOOC']['H'] = intact_bsoc_h

# BOOC: L
growth_rates['BOOC']['L'] = l_booc_0
p_bs['BOOC']['L'] = p_b_booc_0
integrase_activities['BOOC']['L'] = intact_bspc_l

# BOOC: 0
growth_rates['BOOC']['0'] = l_booc_0
p_bs['BOOC']['0'] = p_b_booc_0
integrase_activities['BOOC']['0'] = 0

# OOOC: H
growth_rates['OOOC']['H'] = l_osoc_h
p_bs['OOOC']['H'] = 0
integrase_activities['OOOC']['H'] = intact_osoc_h

# OOOC: L
growth_rates['OOOC']['L'] = l_ospc_h
p_bs['OOOC']['L'] = 0
integrase_activities['OOOC']['L'] = intact_bspc_l

# OOOC: 0
growth_rates['OOOC']['0'] = l_oooc_0
p_bs['OOOC']['0'] = 0
integrase_activities['OOOC']['0'] = 0

In [8]:
# DETERMINISTIC SIMULATION - WITH NO CAT PRESENT, FIND EQUILIBIRA (THERE'LL ONLY BE A SINGLE POSSIBLE SWITCH STATE) AND RECORD VALUES
# we don't care about integrase activity rates here, as the cat gene is not present - so they will all remain at zero

# BSPO
#simulate
par_bspo=par.copy()
par_bspo['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_bspo['func_cat'] = 0
init_conds['p_switch']=0
sol = ode_sim(par_bspo,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_bspo, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_bspo, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_bspo = l  # growth rate
p_b_bspo =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
# record
growth_rates['BSPO']['H'] = l_bspo; growth_rates['BSPO']['L'] = l_bspo; growth_rates['BSPO']['0'] = l_bspo
p_bs['BSPO']['H'] = p_b_bspo; p_bs['BSPO']['L'] = p_b_bspo; p_bs['BSPO']['0'] = p_b_bspo

# OSPO
#simulate
par_ospo=par.copy()
par_ospo['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_ospo['func_cat'] = 0
par_ospo['func_b'] = 0
init_conds['p_switch']=0
sol = ode_sim(par_ospo,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_ospo, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_ospo, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_bspo = l  # growth rate
# record
growth_rates['OSPO']['H'] = l_bspo; growth_rates['OSPO']['L'] = l_bspo; growth_rates['OSPO']['0'] = l_bspo
p_bs['OSPO']['H'] = 0; p_bs['OSPO']['L'] = 0; p_bs['OSPO']['0'] = 0

# BOPO
#simulate
par_bopo=par.copy()
par_bopo['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_bopo['func_cat'] = 0
init_conds['p_switch']=0
sol = ode_sim(par_bopo,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_bopo, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_bopo, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_bopo = l  # growth rate
p_b_bopo =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
# record
growth_rates['BOPO']['H'] = l_bspo; growth_rates['BOPO']['L'] = l_bspo; growth_rates['BOPO']['0'] = l_bspo
p_bs['BOPO']['H'] = p_b_bspo; p_bs['BOPO']['L'] = p_b_bspo; p_bs['BOPO']['0'] = p_b_bspo

# OOPO
#simulate
par_oopo=par.copy()
par_oopo['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_oopo['func_cat'] = 0
par_oopo['func_b'] = 0
par_oopo['func_switch'] = 0
init_conds['p_switch']=0
sol = ode_sim(par_oopo,
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_oopo, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_oopo, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_oopo = l  # growth rate
# record
growth_rates['OOPO']['H'] = l_oopo; growth_rates['OOPO']['L'] = l_oopo; growth_rates['OOPO']['0'] = l_oopo
p_bs['OOPO']['H'] = 0; p_bs['OOPO']['L'] = 0; p_bs['OOPO']['0'] = 0

# BSOO
#simulate
par_bsoo=par.copy()
par_bsoo['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_bsoo['func_cat'] = 0
par_bsoo['func_prot'] = 0
init_conds['p_switch']=0
sol = ode_sim(par_bsoo,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_bsoo, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_bsoo, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_bsoo = l  # growth rate
p_b_bsoo =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
# record
growth_rates['BSOO']['H'] = l_bsoo; growth_rates['BSOO']['L'] = l_bsoo; growth_rates['BSOO']['0'] = l_bsoo
p_bs['BSOO']['H'] = p_b_bsoo; p_bs['BSOO']['L'] = p_b_bsoo; p_bs['BSOO']['0'] = p_b_bsoo

# OSOO
#simulate
par_osoo=par.copy()
par_osoo['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_osoo['func_cat'] = 0
par_osoo['func_b'] = 0
par_osoo['func_prot'] = 0
init_conds['p_switch']=0
sol = ode_sim(par_osoo,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_osoo, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_osoo, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_osoo = l  # growth rate
# record
growth_rates['OSOO']['H'] = l_osoo; growth_rates['OSOO']['L'] = l_osoo; growth_rates['OSOO']['0'] = l_osoo
p_bs['OSOO']['H'] = 0; p_bs['OSOO']['L'] = 0; p_bs['OSOO']['0'] = 0

# BOOO
#simulate
par_booo=par.copy()
par_booo['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_booo['func_cat'] = 0
par_booo['func_switch'] = 0
par_booo['func_prot'] = 0
init_conds['p_switch']=0
sol = ode_sim(par_booo,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_booo, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_booo, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_booo = l  # growth rate
p_b_booo =  xs[-1, circuit_name2pos['p_b']] # burdensome proein concentration
# record
growth_rates['BOOO']['H'] = l_booo; growth_rates['BOOO']['L'] = l_booo; growth_rates['BOOO']['0'] = l_booo
p_bs['BOOO']['H'] = p_b_booo; p_bs['BOOO']['L'] = p_b_booo; p_bs['BOOO']['0'] = p_b_booo

# OOOO
par_oooo=par.copy()
par_oooo['k_sxf'] = 0 # integrase must be disabled to make sure we stay in the genetic state of interest
par_oooo['func_cat'] = 0
par_oooo['func_b'] = 0
par_oooo['func_switch'] = 0
par_oooo['func_prot'] = 0
init_conds['p_switch']=0
sol = ode_sim(par_oooo,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_oooo, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record values
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_oooo, circuit_genes, circuit_miscs, circuit_name2pos)
l=np.float64(ls[-1])
l_oooo = l  # growth rate
# record
growth_rates['OOOO']['H'] = l_oooo; growth_rates['OOOO']['L'] = l_oooo; growth_rates['OOOO']['0'] = l_oooo
p_bs['OOOO']['H'] = 0; p_bs['OOOO']['L'] = 0; p_bs['OOOO']['0'] = 0

In [9]:
# CLEAR MEMORY
del par_bspc, par_ospc, par_bopc, par_oopc, par_bsoc, par_osoc, par_booc, par_oooc
del par_bspo, par_ospo, par_bopo, par_oopo, par_bsoo, par_osoo, par_booo, par_oooo

In [55]:
# DEFINE REMAININING STOCHASTIC TRANSITION RATES
# define gene mutation rates
mutation_rates = [1e-12,1e-12,1e-12,1e-12]
# mutation_rates = [1e-15,1e-15,1e-15,1e-15]

# define switch state transition rates (only those that are non-zero)
transition_rates = {
    'BSPC': {
        'L>H': 1/11.35,
        'H>L': 1/1153.29393262983
    },
    'OSPC': {
        'L>H': 1/5.62,
        'H>L': 1/29994.9997221792
    },
    'BOPC': {
        'H>L': 1/2.601327,
        'L>0': 1/1.49236560946109
    },
    'OOPC': {
        'H>L': 1/2.40654866666666,
        'L>0': 1/1.27014547395815
    },
    'BSOC': {
        'L>H': 1/3.13026566666666
    },
    'OSOC': {
        'L>H': 1/2.70777133333333
    },
    'BOOC': {
        'H>L': 1/2.70764966666666,
        'L>0': 1/1.48274455791453
    },
    'OOOC': {
        'H>L': 1/2.47044833333333,
        'L>0': 1/1.26857090423757
    },
}

In [56]:
# SIMULATE THE POPULATION WITH THE PUNISHER
pop_sim = PopulationSimulator(
    # optional arguments
    growth_rates=growth_rates,
    mutation_rates=mutation_rates,
    transition_rates=transition_rates,
    integrase_activities=integrase_activities,
    p_bs=p_bs
)

# set the simulation parameters
pop_tf = (0, 500)
pop_ts = jnp.arange(pop_tf[0], pop_tf[1], 0.1)
pop_rtol = 1e-6
pop_atol = 1e-6

# set the initial condition - starting with all genes present
x0 = jnp.zeros(pop_sim.num_css).at[pop_sim.cs2p[(1, 1, 1, 1)]['L']].set(1e9)

# x0_bsic = jnp.zeros(pop_sim.num_css).at[pop_sim.cs2p[(1, 1, 1, 1)]['L']].set(1e9/2)
# x0=x0_bsic.at[pop_sim.cs2p[(0, 1, 1, 1)]['L']].set(1e9/2)

pop_sol=pop_ode_sim(x0,pop_tf,pop_ts,pop_rtol,pop_atol,pop_sim)
pop_ts=np.array(pop_sol.ts)
pop_xs=np.array(pop_sol.ys)

In [57]:
# SIMULATE THE POPULATION WITHOUT THE PUNISHER

# set the initial condition - starting with all genes present
x0_no_punisher = jnp.zeros(pop_sim.num_css).at[pop_sim.cs2p[(1, 0, 0, 1)]['0']].set(1e9)

pop_sol_no_punisher=pop_ode_sim(x0_no_punisher,pop_tf,pop_ts,pop_rtol,pop_atol,pop_sim)
pop_ts_no_punisher=np.array(pop_sol_no_punisher.ts)
pop_xs_no_punisher=np.array(pop_sol_no_punisher.ys)

In [58]:
# PLOT THE RESULTS

# FigS1C - genetic states of the population with the punisher
# bkplot.output_file('population_simulation.html')
func_figure=pop_sim.plot_funcstates(pop_ts,pop_xs,
                                    dimensions=(240,180),
                                    tspan=(0,pop_tf[1]))
# cosmetic tweaks to FigS1C
func_figure.output_backend = 'svg'
func_figure.legend.visible=False
func_figure.yaxis.formatter = bkmodels.PrintfTickFormatter(format="%.0e")
func_figure.title='With punisher'
func_figure.xaxis.axis_label_text_font_size = "8pt"
func_figure.xaxis.major_label_text_font_size = "8pt"
func_figure.yaxis.axis_label_text_font_size = "8pt"
func_figure.yaxis.major_label_text_font_size = "8pt"

# FigS1D - genetic states of the population without the punisher
func_figure_no_punisher=pop_sim.plot_funcstates(pop_ts_no_punisher,pop_xs_no_punisher,
                                                  dimensions=(240,180),
                                                    tspan=(0,pop_tf[1]))
# cosmetic tweaks to FigS1D
func_figure_no_punisher.output_backend = 'svg'
func_figure_no_punisher.legend.visible=False
func_figure_no_punisher.yaxis.formatter = bkmodels.PrintfTickFormatter(format="%.0e")
func_figure_no_punisher.title='Without punisher'
func_figure_no_punisher.xaxis.axis_label_text_font_size = "8pt"
func_figure_no_punisher.xaxis.major_label_text_font_size = "8pt"
func_figure_no_punisher.yaxis.axis_label_text_font_size = "8pt"
func_figure_no_punisher.yaxis.major_label_text_font_size = "8pt"


# additional plot: switch states
allfunc_switch_figure=pop_sim.plot_funcstate_switches(pop_ts,pop_xs,
                                                      (1,1,1,1),
                                                      dimensions=(480,360),
                                                      tspan=(0,pop_tf[1]))
# additional plot: burden gene presence
burden_figure = pop_sim.plot_with_burden(pop_ts, pop_xs,
                                         label1='W/punisher',
                                         #
                                         ts2=pop_ts_no_punisher, xs2=pop_xs_no_punisher,
                                         label2='W/o punisher',
                                         #
                                         dimensions=(480, 270),
                                         tspan=(0, pop_tf[1]))

# additional plot: average protein synthesis rate per cell
rate_figure = pop_sim.plot_percell_avg_synth_rate(pop_ts, pop_xs,
                                                 label1='W/punisher',
                                                 #
                                                 ts2=pop_ts_no_punisher, xs2=pop_xs_no_punisher,
                                                 label2='W/o punisher',
                                                 #
                                                 dimensions=(480, 270),
                                                 tspan=(0, pop_tf[1]))

# show plots
# bkplot.show(bklayouts.grid([[func_figure,   allfunc_switch_figure],
#                             [burden_figure, rate_figure]]))
bkplot.show(bklayouts.grid([[func_figure, func_figure_no_punisher]]))

In [61]:
# CONSIDER AND PLOT DIFFERENT MUTATION RATES

# define the range of mutation rates
mutrate_range=np.logspace(-18,-3,6)

# initialise the storage of yields and function durations
yields_pun=np.zeros_like(mutrate_range)
yields_nopun=np.zeros_like(mutrate_range)
funcdurs_pun=np.zeros_like(mutrate_range)
funcdurs_nopun=np.zeros_like(mutrate_range)

# set simulation parameters
# set the simulation parameters
pop_tf_diffmutrates = (0, 500)
pop_savets_diffmutrates = jnp.arange(pop_tf_diffmutrates[0], pop_tf_diffmutrates[1], 0.1)
pop_rtol_diffmutrates = 1e-6
pop_atol_diffmutrates = 1e-6

# loop through the mutation rates
for mutrate_cntr in range(0,len(mutrate_range)):
    print(mutrate_cntr)
    # set the mutation rates
    mutation_rates_diffmutrates = [mutrate_range[mutrate_cntr],mutrate_range[mutrate_cntr],mutrate_range[mutrate_cntr],mutrate_range[mutrate_cntr]]
    # simulate the population with the punisher
    pop_sim_diffmutrates = PopulationSimulator(
        # optional arguments
        growth_rates=growth_rates,
        mutation_rates=mutation_rates_diffmutrates,
        transition_rates=transition_rates,
        integrase_activities=integrase_activities,
        p_bs=p_bs
    )
    x0 = jnp.zeros(pop_sim_diffmutrates.num_css).at[pop_sim_diffmutrates.cs2p[(1, 1, 1, 1)]['L']].set(1e9)
    pop_sol_diffmutrates=pop_ode_sim(x0,
                                     pop_tf_diffmutrates,pop_savets_diffmutrates,
                                     pop_rtol_diffmutrates,pop_atol_diffmutrates,
                                     pop_sim_diffmutrates)
    pop_ts_diffmutrates=np.array(pop_sol_diffmutrates.ts)
    pop_xs_diffmutrates=np.array(pop_sol_diffmutrates.ys)
    # get metrics with the punisher
    avg_synthrates_percell=pop_sim_diffmutrates.percell_avg_synth_rate(pop_ts_diffmutrates,pop_xs_diffmutrates)
    yields_pun[mutrate_cntr]=pop_sim_diffmutrates.percell_yield(pop_ts_diffmutrates,avg_synthrates_percell)
    funcdurs_pun[mutrate_cntr]=pop_sim_diffmutrates.func_duration(pop_ts_diffmutrates,avg_synthrates_percell,share_initial=0.5)
    
    # simulate the population without the punisher
    x0_no_punisher = jnp.zeros(pop_sim_diffmutrates.num_css).at[pop_sim_diffmutrates.cs2p[(1, 0, 0, 1)]['0']].set(1e9)
    pop_sol_diffmutrates_no_punisher=pop_ode_sim(x0_no_punisher,
                                                 pop_tf_diffmutrates,pop_savets_diffmutrates,
                                                 pop_rtol_diffmutrates,pop_atol_diffmutrates,
                                                 pop_sim_diffmutrates)
    pop_ts_diffmutrates_no_punisher=np.array(pop_sol_diffmutrates_no_punisher.ts)
    pop_xs_diffmutrates_no_punisher=np.array(pop_sol_diffmutrates_no_punisher.ys)
    # get metrics without the punisher
    avg_synthrates_percell_no_punisher=pop_sim_diffmutrates.percell_avg_synth_rate(pop_ts_diffmutrates_no_punisher,pop_xs_diffmutrates_no_punisher)
    yields_nopun[mutrate_cntr]=pop_sim_diffmutrates.percell_yield(pop_ts_diffmutrates_no_punisher,avg_synthrates_percell_no_punisher)
    funcdurs_nopun[mutrate_cntr]=pop_sim_diffmutrates.func_duration(pop_ts_diffmutrates_no_punisher,avg_synthrates_percell_no_punisher,share_initial=0.9)

0
1
2
3
4
5


In [62]:
# PLOT METRICS AS FUNCTIONS OF MUTATION RATES

# YIELDS
yield_diffmutrates_figure = bkplot.figure(#title='Yields as functions of mutation rates',
                                            x_axis_type='log',
                                            # y_axis_type='log',
                                            x_axis_label='Gene mutation prob. in each division', 
                                            y_axis_label='Y (total prot. yield), molecules/cell',
                                            frame_width=240,
                                            frame_height=180)
# plot yields with the punisher
yield_diffmutrates_figure.line(mutrate_range, yields_pun,
                                 line_width=2, color='blue',
                                 legend_label='With punisher')
yield_diffmutrates_figure.scatter(mutrate_range, yields_pun, 
                               marker='circle', color='blue',
                               legend_label='With punisher', 
                               line_width=2)
# plot yields without the punisher
yield_diffmutrates_figure.line(mutrate_range, yields_nopun,
 line_width=2, color='red',
                                 legend_label='Without punisher')
yield_diffmutrates_figure.scatter(mutrate_range, yields_nopun,
                                    marker='circle', color='red',
                                    legend_label='Without punisher', 
                                    line_width=2)

# mark the scenario for which the previous plots were made
yield_diffmutrates_figure.vspan(mutation_rates[0],color='black', line_dash='dotted', line_width=1.5)

# x axis ticks
yield_diffmutrates_figure.xaxis.ticker = list(mutrate_range)
# y axis ticks
yield_diffmutrates_figure.yaxis.ticker = list(np.linspace(0, 8e7, 5))
yield_diffmutrates_figure.yaxis.formatter = bkmodels.PrintfTickFormatter(format="%.0e")

# legend settings
yield_diffmutrates_figure.legend.location = 'top_right'
yield_diffmutrates_figure.legend.label_text_font_size = '8pt'
yield_diffmutrates_figure.legend.glyph_width = 10
yield_diffmutrates_figure.legend.padding = 3
yield_diffmutrates_figure.legend.spacing = 3
yield_diffmutrates_figure.legend.margin = 3

# axis label settings
yield_diffmutrates_figure.xaxis.axis_label_text_font_size = "8pt"
yield_diffmutrates_figure.xaxis.major_label_text_font_size = "8pt"
yield_diffmutrates_figure.yaxis.axis_label_text_font_size = "8pt"
yield_diffmutrates_figure.yaxis.major_label_text_font_size = "8pt"

# FUNCTION DURATIONS
funcdur_diffmutrates_figure = bkplot.figure(#title='Function durations as functions of mutation rates',
                                            x_axis_type='log',
                                            x_axis_label='Gene mutation prob. in each division', 
                                            y_axis_label='Function duration, h',
                                            frame_width=240, 
                                            frame_height=180)
# plot function durations with the punisher
funcdur_diffmutrates_figure.line(mutrate_range, funcdurs_pun,
                                 line_width=2, color='blue',
                                 legend_label='With punisher')
funcdur_diffmutrates_figure.scatter(mutrate_range, funcdurs_pun,
                                 marker='circle', color='blue',
                                 legend_label='With punisher', 
                                 line_width=2)
# plot function durations without the punisher
funcdur_diffmutrates_figure.line(mutrate_range, funcdurs_nopun,
                                 line_width=2, color='red',
                                 legend_label='Without punisher')
funcdur_diffmutrates_figure.scatter(mutrate_range, funcdurs_nopun,
                                    marker='circle', color='red',
                                    legend_label='Without punisher', 
                                    line_width=2)

# mark the scenario for which the previous plots were made
funcdur_diffmutrates_figure.vspan(mutation_rates[0],color='black', line_dash='dotted', line_width=1.5)

# x axis ticks
funcdur_diffmutrates_figure.xaxis.ticker = list(mutrate_range)

# legend settings
funcdur_diffmutrates_figure.legend.location = 'top_right'
funcdur_diffmutrates_figure.legend.label_text_font_size = '8pt'
funcdur_diffmutrates_figure.legend.glyph_width = 10
funcdur_diffmutrates_figure.legend.padding = 3
funcdur_diffmutrates_figure.legend.spacing = 3
funcdur_diffmutrates_figure.legend.margin = 3

# axis label settings
funcdur_diffmutrates_figure.xaxis.axis_label_text_font_size = "8pt"
funcdur_diffmutrates_figure.xaxis.major_label_text_font_size = "8pt"
funcdur_diffmutrates_figure.yaxis.axis_label_text_font_size = "8pt"
funcdur_diffmutrates_figure.yaxis.major_label_text_font_size = "8pt"

# show plots
yield_diffmutrates_figure.output_backend = 'svg'
funcdur_diffmutrates_figure.output_backend = 'svg'
bkplot.show(bklayouts.grid([[yield_diffmutrates_figure, funcdur_diffmutrates_figure]]))

In [ ]:
# # PLOT THE RESULTS WITH THE PUNISHER CO-EXPRESSED WITH ESSENTIAL GENES
# 
# # genetic states
# # bkplot.output_file('population_simulation.html')
# func_figure=pop_sim.plot_funcstates(pop_ts_coexp,pop_xs_coexp,
#                                     dimensions=(480,360),
#                                     tspan=(0,pop_tf[1]))
# # switch states
# allfunc_switch_figure=pop_sim.plot_funcstate_switches(pop_ts_coexp,pop_xs_coexp,
#                                                       (0,1,0,1),
#                                                       dimensions=(480,360),
#                                                       tspan=(0,2000))
# # burden gene presence
# burden_figure = pop_sim.plot_with_burden(pop_ts_coexp, pop_xs_coexp,
#                                          label1='W/punisher',
#                                          #
#                                          #ts2=pop_ts_no_punisher, xs2=pop_xs_no_punisher,
#                                          #label2='W/o punisher',
#                                          #
#                                          dimensions=(480, 270),
#                                          tspan=(0, 2000))
# 
# # average protein synthesis rate per cell
# rate_figure = pop_sim.plot_percell_avg_synth_rate(pop_ts_coexp, pop_xs_coexp,
#                                                  label1='W/punisher',
#                                                  #
#                                                  #ts2=pop_ts_no_punisher, xs2=pop_xs_no_punisher,
#                                                  #label2='W/o punisher',
#                                                  #
#                                                  dimensions=(480, 270),
#                                                  tspan=(0, 2000))
# # lifetime metrics
# yield_figure, funcdur_figure = pop_sim.plot_metrics(pop_ts_coexp, pop_xs_coexp,
#                                                  share_initial=0.9,
#                                                  #
#                                                  #ts2=pop_ts_no_punisher, xs2=pop_xs_no_punisher,
#                                                  label1='W/punisher',
#                                                  #label2='W/o punisher',
#                                                  #
#                                                  dimensions=(480, 270),
#                                                  tspan=(0, pop_tf[1]))
# 
# 
# # show plots
# # bkplot.show(bklayouts.grid([[func_figure,   None],
# #                             [burden_figure, rate_figure],
# #                             [yield_figure,  funcdur_figure]]))

In [ ]:
# # SIMULATE THE POPULATION WITH THE PUNISHER CO-EXPRESSED WITH ESSENTIAL GENES
# # cells without the punisher will be unablke to grow
# growth_rates_coexp = copy.deepcopy(growth_rates)
# for func in growth_rates_coexp.keys():
#     if(func[1]=='O'):
#         growth_rates_coexp[func]['H'] = 0
#         growth_rates_coexp[func]['L'] = 0
#         growth_rates_coexp[func]['0'] = 0
#         
# pop_sim_coexp = PopulationSimulator(
#     # optional arguments
#     growth_rates=growth_rates_coexp,
#     mutation_rates=mutation_rates,
#     transition_rates=transition_rates,
#     integrase_activities=integrase_activities,
#     p_bs=p_bs
# )
# 
# # set the initial condition - starting with all genes present
# x0 = jnp.zeros(pop_sim_coexp.num_css).at[pop_sim_coexp.cs2p[(1, 1, 1, 1)]['L']].set(1e9)
# 
# pop_sol_coexp=pop_ode_sim(x0,(0,2000),np.arange(0,2000,10),pop_rtol,pop_atol,pop_sim_coexp)
# 
# pop_ts_coexp=np.array(pop_sol_coexp.ts)
# pop_xs_coexp=np.array(pop_sol_coexp.ys)